In [1]:
!pip install sumy rouge-score datasets -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install openpyxl
import openpyxl
from openpyxl import Workbook

In [3]:
wb = Workbook()

In [4]:
import torch
from datasets import load_dataset
from rouge_score import rouge_scorer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from collections import Counter
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")
texts = dataset['article'][6:11]
references = dataset['highlights'][6:11]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [6]:
def prepare_for_sumy(text):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    return parser

In [7]:
def summarize_with_model(text, model_name, sentences_count=5):
    parser = prepare_for_sumy(text)

    if model_name == "textrank":
        summarizer = TextRankSummarizer()
    elif model_name == "lsa":
        summarizer = LsaSummarizer()
    elif model_name == "frequency":
        summarizer = custom_frequency_summarizer(text, sentences_count)
        return summarizer
    else:
        raise ValueError(f"Model {model_name} not supported")

    summary = summarizer(parser.document, sentences_count)
    return "\n".join([str(sentence) for sentence in summary])


In [8]:
def custom_frequency_summarizer(text, sentences_count):
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text.lower())

    # Compute word frequencies
    word_frequencies = Counter(words)
    max_frequency = max(word_frequencies.values())

    # Normalize word frequencies
    word_frequencies = {word: freq / max_frequency for word, freq in word_frequencies.items()}

    # Score sentences by summing the frequency of the words
    sentence_scores = {}
    for i, sentence in enumerate(sentences):
        sentence_words = nltk.word_tokenize(sentence.lower())
        sentence_score = sum(word_frequencies.get(word, 0) for word in sentence_words)
        sentence_scores[i] = sentence_score

    # Sort and select the top `sentences_count` sentences
    sorted_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    top_sentences = [sentences[i] for i in sorted_sentences[:sentences_count]]

    return " ".join(top_sentences)

In [9]:
def calculate_rouge(summaries, references,model_name):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = { 'rouge1': 0, 'rouge2': 0, 'rougeL': 0 }
    i=0
    for summary, reference in zip(summaries, references):
        i+=1

        score = scorer.score(reference, summary)
        print(f'rouge1 score of summary{i}:{score["rouge1"].fmeasure}')
        print(f'rouge2 score of summary{i}:{score["rouge2"].fmeasure}')
        print(f'rougeL score of summary{i}:{score["rougeL"].fmeasure}')
        scores['rouge1'] += score['rouge1'].fmeasure
        scores['rouge2'] += score['rouge2'].fmeasure
        scores['rougeL'] += score['rougeL'].fmeasure
        ws.cell(row=i+1, column=4, value=scores['rouge1'])
        ws.cell(row=i+1, column=5, value=scores['rouge2'])
        ws.cell(row=i+1, column=6, value=scores['rougeL'])

    num_samples = len(summaries)
    scores = { k: v / num_samples for k, v in scores.items() }
    for j, val in enumerate(scores.values()):
        ws.cell(row=i+2, column=j+4, value=val)
    return scores

In [10]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(summaries, references, model_name):

    bleu_scores = []
    smoothie = SmoothingFunction().method4
    i=0
    for summary, reference in zip(summaries, references):
        i+=1
        reference_tokens = [reference.split()]
        summary_tokens = summary.split()

        score = sentence_bleu(reference_tokens, summary_tokens,weights=(0.5,0.5,0,0) ,smoothing_function=smoothie)
        bleu_scores.append(score)
        print(f"BLEU score of summary{i}: {score}")
        ws.cell(row=i+1, column=7, value=score)

    average_bleu = sum(bleu_scores) / len(bleu_scores)
    ws.cell(row=i+2, column=7, value=average_bleu)
    return average_bleu

In [11]:
for model_name in ["textrank", "lsa", "frequency"]:
    print(f"Running summarization with {model_name}...")

    # Generate summaries
    summaries = [summarize_with_model(text, model_name) for text in texts]
    for i, summary in enumerate(summaries):
        print(f"\nSummary {i+1} ({model_name}):")
        print(summary)
    # creatin new sheet
    ws = wb.create_sheet(title=model_name)
    column=['Text',"reference Summary","Summary","rouge 1","rouge 2","rouge L","Bleu score"]
    for i,val in enumerate(column):
        ws.cell(row=1, column=i+1, value=val)
    for i, text in enumerate(texts):
        ws.cell(row=i+2, column=1, value=text)
    for i, summary in enumerate(summaries):
        ws.cell(row=i+2, column=3, value=summary)
    for i, ref in enumerate(references):
        ws.cell(row=i+2, column=2, value=ref)

    # Calculate ROUGE scores
    scores = calculate_rouge(summaries, references,model_name)
    print(f"ROUGE Scores for {model_name}:", scores)
    bleu_score = calculate_bleu(summaries, references,model_name)
    print(f"Average BLEU score: {bleu_score}")


Running summarization with textrank...

Summary 1 (textrank):
"The dark trend of governments using the death penalty in a futile attempt to tackle real or imaginary threats to state security and public safety was stark last year," said Salil Shetty, Amnesty's Secretary General in a release.
The report, "Death Sentences and Executions 2014," cites the example of Pakistan lifting a six-year moratorium on the execution of civilians following the horrific attack on a school in Peshawar in December.
The resumption of the use of capital punishment in Belarus -- the only country in Europe and Central Asia to execute people -- after a two year hiatus spoiled an near-universal decrease in countries using the death penalty by region.
The United States has the dubious distinction of being the only country in the Americas to conduct executions, but the number of convicts put to death here fell slightly, from 39 in 2013 to 35 in 2014.
"In the majority of countries where people were sentenced to dea

In [12]:
wb.save('/content/drive/My Drive/summary_results.xlsx')